# Arquitecturas híbridas

## Librerías

In [8]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, cohen_kappa_score, confusion_matrix, classification_report)
from tqdm import tqdm

## Fusión temprana

In [10]:
# --- CONFIGURACIÓN ---
# Detectamos si tienes GPU para que la extracción de características vuele
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Usando dispositivo: {device}")

# Rutas
csv_path = 'Datos/metadata.csv'
images_dir = 'Datos/images'

# ==========================================
# 1. PREPARACIÓN DE LA CNN (ResNet101)
# ==========================================
print("\n🧠 Cargando ResNet101 preentrenada...")

# Función para obtener el modelo extractor de características
def get_feature_extractor():
    # Cargamos ResNet101 con pesos de ImageNet
    model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
    
    # TRUCO: Quitamos la última capa (fc) que es la que clasifica (perro, gato...)
    # La sustituimos por una identidad para que pase el vector directo
    # ResNet101 saca un vector de 2048 características antes de la clasificación
    model.fc = nn.Identity()
    
    model.to(device)
    model.eval() # Modo evaluación (congela dropout, etc)
    return model

cnn_extractor = get_feature_extractor()

# Transformaciones necesarias para que ResNet entienda las fotos
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ==========================================
# 2. CARGA DE DATOS E IMÁGENES
# ==========================================
class SkinLesionDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Reconstruimos la ruta: dataset_final/images/CLASE/FOTO.png
        img_name = row['img_id']
        label_name = row['diagnostic']
        img_path = os.path.join(self.root_dir, label_name, img_name)
        
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            # Si falla una imagen, creamos una negra (no debería pasar si limpiamos bien)
            image = Image.new('RGB', (224, 224))
            
        if self.transform:
            image = self.transform(image)
            
        return image

# Leemos el CSV
df = pd.read_csv(csv_path)
print(f"📂 Dataset cargado: {len(df)} muestras")

# Dataset y DataLoader para ir por lotes
dataset = SkinLesionDataset(df, images_dir, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False) # Shuffle False para mantener orden con CSV!


🚀 Usando dispositivo: cpu

🧠 Cargando ResNet101 preentrenada...
📂 Dataset cargado: 802 muestras


In [11]:

# ==========================================
# 3. EXTRACCIÓN DE VECTORES (EMBEDDINGS)
# ==========================================
print("\n📸 Extrayendo características visuales (Esto puede tardar un poco)...")

features_list = []

with torch.no_grad(): # No necesitamos gradientes, solo inferencia
    for images in tqdm(dataloader):
        images = images.to(device)
        # Pasamos las fotos por la ResNet
        outputs = cnn_extractor(images)
        # Pasamos a CPU y numpy
        features_list.append(outputs.cpu().numpy())

# Unimos todos los bloques en una gran matriz de características
# Tamaño esperado: (N_muestras, 2048)
image_features = np.concatenate(features_list, axis=0)
print(f"✅ Extracción completada. Dimensión de features visuales: {image_features.shape}")

# Convertimos a DataFrame para unirlo bonito
feat_cols = [f'img_feat_{i}' for i in range(image_features.shape[1])]
df_features = pd.DataFrame(image_features, columns=feat_cols)



📸 Extrayendo características visuales (Esto puede tardar un poco)...


100%|██████████| 26/26 [01:41<00:00,  3.92s/it]

✅ Extracción completada. Dimensión de features visuales: (802, 2048)


In [12]:

# ==========================================
# 4. PREPARACIÓN DATOS TABULARES (Limpieza)
# ==========================================
print("\n⚙️ Procesando datos tabulares...")

# Hacemos copia para no tocar el original
df_tab = df.copy()

# Variables a ignorar
ignore_cols = ['patient_id', 'lesion_id', 'img_id', 'diagnostic', 'biopsed', 'path'] # path si existiera
cols_to_use = [c for c in df_tab.columns if c not in ignore_cols]

X_tab = df_tab[cols_to_use]
y = df_tab['diagnostic']

# Limpieza (Rellenar nulos y One-Hot Encoding)
# 1. Rellenar
for col in X_tab.columns:
    if X_tab[col].dtype == 'object':
        X_tab[col] = X_tab[col].fillna('DESCONOCIDO')
    else:
        X_tab[col] = X_tab[col].fillna(X_tab[col].mean())

# 2. Convertir texto a números (One-Hot)
X_tab = pd.get_dummies(X_tab, drop_first=True)

print(f"   - Features Tabulares: {X_tab.shape[1]} columnas")

# ==========================================
# 5. FUSIÓN (CONCATENACIÓN)
# ==========================================
print("\n🔗 FUSIONANDO (Early Fusion)...")

# Reseteamos índices para evitar problemas al concatenar
X_tab.reset_index(drop=True, inplace=True)
df_features.reset_index(drop=True, inplace=True)

# Unimos horizontalmente: [Datos Tabulares | Vector ResNet]
X_final = pd.concat([X_tab, df_features], axis=1)

print(f"✨ Dataset Híbrido Final: {X_final.shape[0]} filas x {X_final.shape[1]} columnas")
# Codificamos la etiqueta (Target) a números (0, 1, 2...)
le = LabelEncoder()
y_encoded = le.fit_transform(y)



⚙️ Procesando datos tabulares...
   - Features Tabulares: 66 columnas

🔗 FUSIONANDO (Early Fusion)...
✨ Dataset Híbrido Final: 802 filas x 2114 columnas


C:\Users\pedro\AppData\Local\Temp\ipykernel_12936\1936509309.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tab[col] = X_tab[col].fillna('DESCONOCIDO')
C:\Users\pedro\AppData\Local\Temp\ipykernel_12936\1936509309.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_tab[col] = X_tab[col].fillna(X_tab[col].mean())


In [14]:

# ==========================================
# 6. ENTRENAMIENTO Y EVALUACIÓN
# ==========================================
print("\n🏋️ Entrenando Gradient Boosting (Esto es potente)...")

X_train, X_test, y_train, y_test = train_test_split(X_final, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

# Usamos GradientBoostingClassifier de Sklearn (robusto y potente)
# Si quisieras XGBoost específico: import xgboost as xgb; clf = xgb.XGBClassifier(...)
#clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
#clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
clf = RandomForestClassifier(n_estimators=400, random_state=42)
clf.fit(X_train, y_train)

# Predicción
y_pred = clf.predict(X_test)

# ==========================================
# 7. MÉTRICAS SOLICITADAS
# ==========================================
print("\n📊 --- RESULTADOS DE LA EVALUACIÓN ---")

# Calculamos métricas
acc = accuracy_score(y_test, y_pred)
# 'weighted' calcula la media ponderada por el número de muestras de cada clase (ideal para multiclass)
prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
kappa = cohen_kappa_score(y_test, y_pred)
conf_mat = confusion_matrix(y_test, y_pred)

print(f"✅ Accuracy (Exactitud):  {acc:.4f}")
print(f"✅ Precision (Precisión): {prec:.4f}")
print(f"✅ Recall (Sensibilidad): {rec:.4f}")
print(f"✅ F1-Score:              {f1:.4f}")
print(f"✅ Kappa de Cohen:        {kappa:.4f}")

print("\n🔍 Matriz de Confusión:")
print(conf_mat)

print("\n📋 Reporte Detallado:")
# Recuperamos los nombres reales de las clases para el reporte
target_names = le.inverse_transform(sorted(list(set(y_test))))
print(classification_report(y_test, y_pred, target_names=target_names))


🏋️ Entrenando Gradient Boosting (Esto es potente)...

📊 --- RESULTADOS DE LA EVALUACIÓN ---
✅ Accuracy (Exactitud):  0.5342
✅ Precision (Precisión): 0.5162
✅ Recall (Sensibilidad): 0.5342
✅ F1-Score:              0.5217
✅ Kappa de Cohen:        0.4275

🔍 Matriz de Confusión:
[[14  5  0  0 11  0]
 [ 5 19  0  0  6  0]
 [ 3  1  0  3  3  1]
 [ 1  3  0 21  1  4]
 [ 7 11  0  1 11  0]
 [ 1  2  0  4  2 21]]

📋 Reporte Detallado:
              precision    recall  f1-score   support

         ACK       0.45      0.47      0.46        30
         BCC       0.46      0.63      0.54        30
         MEL       0.00      0.00      0.00        11
         NEV       0.72      0.70      0.71        30
         SCC       0.32      0.37      0.34        30
         SEK       0.81      0.70      0.75        30

    accuracy                           0.53       161
   macro avg       0.46      0.48      0.47       161
weighted avg       0.52      0.53      0.52       161



c:\Users\pedro\anaconda4\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pedro\anaconda4\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pedro\anaconda4\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Fusión tardía (Stacking)

In [15]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, cohen_kappa_score, confusion_matrix, classification_report)
from tqdm import tqdm


In [17]:

# --- CONFIGURACIÓN ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Usando dispositivo: {device}")

csv_path = 'Datos/metadata.csv'
images_dir = 'Datos/images'

# ==========================================
# 1. PREPARACIÓN DE DATOS (SPLIT 3 VÍAS)
# ==========================================
print("\n✂️ Dividiendo datos en Train (Modelos Base), Val (Meta-Modelo) y Test (Final)...")
df = pd.read_csv(csv_path)

# Codificamos las etiquetas a números (0, 1, 2...)
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['diagnostic'])
num_classes = len(le.classes_)
print(f"Clases detectadas: {le.classes_}")

# Split 1: Separamos Test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    df, df['label_encoded'], test_size=0.2, random_state=42, stratify=df['label_encoded']
)

# Split 2: Del resto, separamos Validación (20% del total original aprox)
# Train será el 60% del total, Val el 20%, Test el 20%
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print(f"   - Train set: {len(X_train)} (Para entrenar ResNet, XGB, RF)")
print(f"   - Val set:   {len(X_val)}   (Para entrenar al MLP Stacker)")
print(f"   - Test set:  {len(X_test)}  (Para evaluación final)")


🚀 Usando dispositivo: cpu

✂️ Dividiendo datos en Train (Modelos Base), Val (Meta-Modelo) y Test (Final)...
Clases detectadas: ['ACK' 'BCC' 'MEL' 'NEV' 'SCC' 'SEK']
   - Train set: 480 (Para entrenar ResNet, XGB, RF)
   - Val set:   161   (Para entrenar al MLP Stacker)
   - Test set:  161  (Para evaluación final)


In [19]:

# ==========================================
# 2. MODELO DE IMAGEN (RESNET101 - FINE TUNING)
# ==========================================
print("\n🧠 Configurando ResNet101 para Fine-Tuning...")

# Dataset Personalizado
class SkinDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.root_dir, row['diagnostic'], row['img_id'])
        label = row['label_encoded']
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224)) # Fallback
        
        if self.transform:
            image = self.transform(image)
        return image, label

# Transformaciones
train_transforms = transforms.Compose([
    transforms.Resize(256), transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_test_transforms = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# DataLoaders
train_loader = DataLoader(SkinDataset(X_train, images_dir, train_transforms), batch_size=32, shuffle=True)
val_loader = DataLoader(SkinDataset(X_val, images_dir, val_test_transforms), batch_size=32, shuffle=False)
test_loader = DataLoader(SkinDataset(X_test, images_dir, val_test_transforms), batch_size=32, shuffle=False)

# Definir Modelo
resnet = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
# Congelamos capas base (opcional, para ir más rápido)
for param in resnet.parameters():
    param.requires_grad = False
# Cambiamos la capa final para clasificar nuestras clases
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)
resnet = resnet.to(device)

# Entrenamiento Rápido de ResNet
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("🏋️ Entrenando ResNet (Fine-Tuning de la última capa)...")
resnet.train()
epochs = 5 # Pocas épocas para demostración
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs, labels = inputs.to(device), labels.to(device, dtype=torch.long)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

# Función para obtener PROBABILIDADES
def get_probs(model, loader):
    model.eval()
    probs_list = []
    with torch.no_grad():
        for inputs, _ in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            # Aplicamos Softmax para tener probabilidades (0 a 1)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            probs_list.append(probs.cpu().numpy())
    return np.concatenate(probs_list)

print("📸 Generando predicciones de ResNet para Val y Test...")
resnet_probs_val = get_probs(resnet, val_loader)
resnet_probs_test = get_probs(resnet, test_loader)



🧠 Configurando ResNet101 para Fine-Tuning...
🏋️ Entrenando ResNet (Fine-Tuning de la última capa)...


Epoch 5/5: 100%|██████████| 15/15 [01:15<00:00,  5.05s/it]


📸 Generando predicciones de ResNet para Val y Test...


In [20]:

# ==========================================
# 3. MODELOS TABULARES (XGBOOST y RANDOM FOREST)
# ==========================================
print("\n📊 Entrenando Modelos Tabulares...")

# Preprocesamiento Tabular
def process_tabular(df_subset):
    df_proc = df_subset.drop(columns=['patient_id', 'lesion_id', 'img_id', 'diagnostic', 'path', 'label_encoded', 'biopsed'], errors='ignore')
    # Rellenar y One-Hot
    for col in df_proc.columns:
        if df_proc[col].dtype == 'object':
            df_proc[col] = df_proc[col].fillna('DESCONOCIDO')
        else:
            df_proc[col] = df_proc[col].fillna(df_proc[col].mean())
    return pd.get_dummies(df_proc, drop_first=True)

# Procesamos todos juntos para alinear columnas, luego separamos
df_all_tab = pd.concat([X_train, X_val, X_test])
df_all_proc = process_tabular(df_all_tab)

# Recuperamos los splits procesados
tab_train = df_all_proc.iloc[:len(X_train)]
tab_val = df_all_proc.iloc[len(X_train):len(X_train)+len(X_val)]
tab_test = df_all_proc.iloc[len(X_train)+len(X_val):]

# 3.1 Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(tab_train, y_train)
rf_probs_val = rf.predict_proba(tab_val)
rf_probs_test = rf.predict_proba(tab_test)

# 3.2 XGBoost
xgb = XGBClassifier(eval_metric='mlogloss', use_label_encoder=False)
xgb.fit(tab_train, y_train)
xgb_probs_val = xgb.predict_proba(tab_val)
xgb_probs_test = xgb.predict_proba(tab_test)

print("✅ Modelos tabulares entrenados y predicciones generadas.")



📊 Entrenando Modelos Tabulares...


c:\Users\pedro\anaconda4\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:27:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ Modelos tabulares entrenados y predicciones generadas.


In [21]:

# ==========================================
# 4. STACKING (FUSIÓN TARDÍA CON MLP)
# ==========================================
print("\n🔗 Construyendo el Meta-Modelo (Stacking con MLP)...")

# Construimos la entrada para el meta-modelo concatenando las probabilidades
# Input = [Probs ResNet (6 cols) | Probs RF (6 cols) | Probs XGB (6 cols)]
X_stack_val = np.hstack([resnet_probs_val, rf_probs_val, xgb_probs_val])
X_stack_test = np.hstack([resnet_probs_test, rf_probs_test, xgb_probs_test])

print(f"   - Dimensiones entrada Stacking: {X_stack_val.shape}")

# Usamos un MLPClassifier (Perceptrón Multicapa) como meta-aprendiz
# Es el "Juez" que decide a quién creer
meta_model = MLPClassifier(hidden_layer_sizes=(32, 16), activation='relu', solver='adam', max_iter=500, random_state=42)
meta_model.fit(X_stack_val, y_val)

print("🔮 Prediciendo resultado final con el Stacking...")
y_final_pred = meta_model.predict(X_stack_test)



🔗 Construyendo el Meta-Modelo (Stacking con MLP)...
   - Dimensiones entrada Stacking: (161, 18)
🔮 Prediciendo resultado final con el Stacking...


c:\Users\pedro\anaconda4\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [22]:

# ==========================================
# 5. EVALUACIÓN FINAL
# ==========================================
print("\n🏆 --- RESULTADOS FINALES (STACKING) ---")

# Calculamos métricas
acc = accuracy_score(y_test, y_final_pred)
prec = precision_score(y_test, y_final_pred, average='weighted', zero_division=0)
rec = recall_score(y_test, y_final_pred, average='weighted')
f1 = f1_score(y_test, y_final_pred, average='weighted')
kappa = cohen_kappa_score(y_test, y_final_pred)
conf_mat = confusion_matrix(y_test, y_final_pred)

print(f"✅ Accuracy:        {acc:.4f}")
print(f"✅ Precision:       {prec:.4f}")
print(f"✅ Recall:          {rec:.4f}")
print(f"✅ F1-Score:        {f1:.4f}")
print(f"✅ Kappa de Cohen:  {kappa:.4f}")

print("\n🔍 Matriz de Confusión:")
print(conf_mat)

print("\n📋 Reporte por Clase:")
print(classification_report(y_test, y_final_pred, target_names=le.classes_))


🏆 --- RESULTADOS FINALES (STACKING) ---
✅ Accuracy:        0.6708
✅ Precision:       0.6805
✅ Recall:          0.6708
✅ F1-Score:        0.6701
✅ Kappa de Cohen:  0.5980

🔍 Matriz de Confusión:
[[23  4  0  1  1  1]
 [ 6 17  0  0  6  1]
 [ 0  0  6  2  1  2]
 [ 3  1  0 24  0  2]
 [ 5  8  0  0 17  0]
 [ 0  2  1  6  0 21]]

📋 Reporte por Clase:
              precision    recall  f1-score   support

         ACK       0.62      0.77      0.69        30
         BCC       0.53      0.57      0.55        30
         MEL       0.86      0.55      0.67        11
         NEV       0.73      0.80      0.76        30
         SCC       0.68      0.57      0.62        30
         SEK       0.78      0.70      0.74        30

    accuracy                           0.67       161
   macro avg       0.70      0.66      0.67       161
weighted avg       0.68      0.67      0.67       161

